# Introduction to ByteCore Byte

Welcome to the ByteCore Byte notebook! This is a specialized version of the ByteCore CPU that operates within the ByteCore Emulator. Designed uniquely using the ByteCore Assembly language, it features a simplified addressing system that uses only 1 byte instead of 2. This change reduces the available memory from 64KB to just 256 bytes, providing a compact environment for exploring the capabilities of ByteCore.

## What's in this Notebook?

This notebook is structured to facilitate an interactive exploration of ByteCore Byte:
- **Main ByteCore Byte Source Code:** The first Python cell contains the entire source code necessary for setting up ByteCore Byte.
- **Example Programs:** Followed by two additional code cells that include:
  - **Simple Example Program:** A straightforward program designed to demonstrate basic operations.
  - **Advanced Example Program:** A more complex program that showcases advanced features.

## Running the Examples

To run the examples included in this notebook:
1. Ensure you've followed the setup instructions to properly configure your environment.
2. Open the `bytecorebyte.ipynb` notebook in Jupyter Lab.
3. Click 'Run All' from the Jupyter Lab interface to execute the cells sequentially.

Each program execution is followed by Python assertions that check and verify the CPU's state upon halting, ensuring that each step behaves as expected.

Enjoy exploring ByteCore Byte and seeing assembly language principles in action within a controlled CPU simulation!

## Main ByteCore Byte Source Code

In [1]:
from bytecorecompiler.compiler import Compiler
from bytecore.emulator import ByteCore

bytecorebyte_bytecore_assembly = """
;--------------------
; # ByteCore Byte
;--------------------
; Jump to start of program loop
00 00 JMP
00 01 02
00 02 10

;--------------------
; ## Variables and OPCODES
;--------------------
01 00 00    ; Value  0 ; HALT
01 01 01    ; Value  1 ; LOAD
01 02 02    ; Value  2 ; STORE
01 04 04    ; Value  4 ; ADD
01 08 08    ; Value  8 ; SUB
01 10 10    ; Value 16 ; JMP
01 20 20    ; Value 32 ; JZ

;--------------------
; ## Program loop
;--------------------
; State
02 00 00    ; Accumulator
02 01 00    ; Opcode
02 02 00    ; Data

;--------------------
; ### Start of program loop
;--------------------
; Get PC pointer
02 10 LOAD
02 11 21
02 12 00
02 13 STORE ; Store pointer to PC pointer
02 14 02
02 15 18

;--------------------
; ### Get PC value and store it in state
;--------------------
; Get PC value from pointer
02 16 LOAD
02 17 FF    ; FIXED
02 18 FF
02 19 STORE
02 1A 02
02 1B 01

;--------------------
; ### Check if OPCODE is HALT 00
;--------------------
02 1C JZ
02 1D 02
02 1E 22
02 1F JMP   ; Was not HALT, jump to increment PC
02 20 02
02 21 50

; Execute HALT
02 22 LOAD  ; Get PC pointer
02 23 21
02 24 00
02 25 STORE ; Store pointer to PC pointer
02 26 02
02 27 2D
02 28 LOAD  ; Load accumulator
02 29 02
02 2A 00
02 2B JMP   ; HALT at PC
02 2C FF    ; FIXED
02 2D FF

;--------------------
; ### Increment PC
;--------------------
; Return address variables
02 40 02
02 41 5F

; Copy return address to PC
02 50 LOAD
02 51 02
02 52 40
02 53 STORE
02 54 21
02 55 03
02 56 LOAD
02 57 02
02 58 41
02 59 STORE
02 5A 21
02 5B 04

; Call subroutine: ++PC
02 5C JMP
02 5D 21
02 5E 05

;--------------------
; ### Get DATA value and store it in state
;--------------------
; Get PC value from pointer
02 5F LOAD
02 60 21
02 61 00
02 62 STORE
02 63 02
02 64 67

; Get PC value from pointer
02 65 LOAD
02 66 FF    ; FIXED
02 67 FF
02 68 STORE
02 69 02
02 6A 02
02 6B JMP
02 6C 02
02 6D 70

;--------------------
; ### Check if OPCODE is LOAD 01
;--------------------
02 70 LOAD
02 71 02
02 72 01
02 73 SUB   ; Accumulator - 1
02 74 01
02 75 01
02 76 JZ    ; Is opcode LOAD
02 77 02
02 78 7C
02 79 JMP   ; Is not opcode LOAD, try next
02 7A 02
02 7B 90

; Execute LOAD
02 7C LOAD
02 7D 02
02 7E 02
02 7F STORE
02 80 02
02 81 84
02 82 LOAD  ; Resolve pointer
02 83 FF    ; FIXED
02 84 FF
02 85 STORE ; Store to accumulator
02 86 02
02 87 00
02 88 JMP   ; Jump to increment PC
02 89 03
02 8A 70

;--------------------
; ### Check if OPCODE is STORE 02
;--------------------
02 90 LOAD
02 91 02
02 92 01
02 93 SUB   ; Accumulator - 2
02 94 01
02 95 02
02 96 JZ    ; Is opcode STORE
02 97 02
02 98 9C
02 99 JMP   ; Is not opcode STORE, try next
02 9A 02
02 9B B0

; Execute STORE
02 9C LOAD
02 9D 02
02 9E 02
02 9F STORE
02 A0 02
02 A1 A7
02 A2 LOAD  ; Load from accumulator
02 A3 02
02 A4 00
02 A5 STORE ; Resolve pointer
02 A6 FF    ; FIXED
02 A7 FF
02 A8 JMP   ; Jump to increment PC
02 A9 03
02 AA 70

;--------------------
; ### Check if OPCODE is ADD 04
;--------------------
02 B0 LOAD
02 B1 02
02 B2 01
02 B3 SUB   ; Accumulator - 4
02 B4 01
02 B5 04
02 B6 JZ    ; Is opcode ADD
02 B7 02
02 B8 BC
02 B9 JMP   ; Is not opcode ADD, try next
02 BA 02
02 BB D0

; Execute ADD
02 BC LOAD  ; Load data pointer stored in state
02 BD 02
02 BE 02
02 BF STORE ; Store data pointer
02 C0 02
02 C1 C7
02 C2 LOAD  ; Load accumulator from state
02 C3 02
02 C4 00
02 C5 ADD   ; Resolve pointer
02 C6 FF    ; FIXED
02 C7 FF
02 C8 STORE ; Store back to accumulator stored in state
02 C9 02
02 CA 00
02 CB JMP   ; Jump to increment PC
02 CC 03
02 CD 70

;--------------------
; ### Check if OPCODE is SUB 08
;--------------------
02 D0 LOAD
02 D1 02
02 D2 01
02 D3 SUB   ; Accumulator - 8
02 D4 01
02 D5 08
02 D6 JZ    ; Is opcode SUB
02 D7 02
02 D8 DC
02 D9 JMP   ; Is not opcode SUB, try next
02 DA 02
02 DB F0

; Execute SUB
02 DC LOAD  ; Load data pointer stored in state
02 DD 02
02 DE 02
02 DF STORE ; Store data pointer
02 E0 02
02 E1 E7
02 E2 LOAD  ; Load accumulator from state
02 E3 02
02 E4 00
02 E5 SUB   ; Resolve pointer
02 E6 FF    ; FIXED
02 E7 FF
02 E8 STORE ; Store back to accumulator stored in state
02 E9 02
02 EA 00
02 EB JMP   ; Jump to increment PC
02 EC 03
02 ED 70

;--------------------
; ### Check if OPCODE is JMP 10
;--------------------
02 F0 LOAD
02 F1 02
02 F2 01
02 F3 SUB   ; Accumulator - 16
02 F4 01
02 F5 10
02 F6 JZ    ; Is opcode JMP
02 F7 02
02 F8 FC
02 F9 JMP   ; Is not opcode JMP, try next
02 FA 03
02 FB 10

; Execute JMP
02 FC LOAD  ; Load data pointer stored in state
02 FD 02
02 FE 02
02 FF STORE ; Store to PC
03 00 21
03 01 00
03 02 JMP   ; Jump to loop to top (do not increment PC)
03 03 03
03 04 80

;--------------------
; ### Check if OPCODE is JZ 20
;--------------------
03 10 LOAD
03 11 02
03 12 01
03 13 SUB   ; Accumulator - 32
03 14 01
03 15 20
03 16 JZ    ; Is opcode JZ
03 17 03
03 18 1C
03 19 JMP   ; Is not opcode JZ, jump to invalid opcode
03 1A 03
03 1B 40

; Execute JZ
03 1C LOAD  ; Load data pointer stored in state
03 1D 02
03 1E 00
03 1F JZ    ; Is zero, execute JMP
03 20 02
03 21 FC
03 22 JMP   ; Is not zero, jump to increment PC
03 23 03
03 24 70

;--------------------
; ### Invalid Opcode
;--------------------
; Return address variables
03 30 03
03 31 4F

; Copy return address to PC
03 40 LOAD
03 41 03
03 42 30
03 43 STORE
03 44 21
03 45 03
03 46 LOAD
03 47 03
03 48 31
03 49 STORE
03 4A 21
03 4B 04

; Call subroutine: --PC
03 4C JMP
03 4D 21
03 4E 08

03 4F JMP   ; Jump to HALT
03 50 02
03 51 22

;--------------------
; ### Increment PC
;--------------------
; Return address variables
03 60 03
03 61 80

; Copy return address to PC
03 70 LOAD
03 71 03
03 72 60
03 73 STORE
03 74 21
03 75 03
03 76 LOAD
03 77 03
03 78 61
03 79 STORE
03 7A 21
03 7B 04

; Call subroutine: ++PC
03 7C JMP
03 7D 21
03 7E 05

;--------------------
; ### Loop to top
;--------------------
03 80 JMP
03 81 02
03 82 10

;--------------------
; ## Subroutines
;--------------------

;--------------------
; ### PC - Program Counter
;--------------------
21 00 00    ; PC variable
21 01 01    ; Value  1
; Return address
21 02 JMP
21 03 FE
21 04 FF
; Subroutine ++PC
21 05 JMP
21 06 21
21 07 10
; Subroutine --PC
21 08 JMP
21 09 21
21 0A 20

;--------------------
; #### Subroutine ++PC
;--------------------
21 10 LOAD
21 11 21
21 12 00
21 13 ADD
21 14 21
21 15 01
21 16 STORE
21 17 21
21 18 00

21 19 JMP
21 1A 21
21 1B 02

;--------------------
; #### Subroutine --PC
;--------------------
21 20 LOAD
21 21 21
21 22 00
21 23 SUB
21 24 21
21 25 01
21 26 STORE
21 27 21
21 28 00

21 29 JMP
21 2A 21
21 2B 02
;--------------------
"""

## Simple Example Program

In [2]:
simple_program_bytecore_assembly = """
FF 00 01 ; LOAD
FF 01 0A
FF 02 04 ; ADD
FF 03 0B
FF 04 02 ; STORE
FF 05 FF
FF 06 00 ; HALT
FF 0A 14 ; 20
FF 0B 1E ; 30
"""

# Arrange
expected = 50
bytecore_assembly = '\n'.join(
    [bytecorebyte_bytecore_assembly, simple_program_bytecore_assembly])

# Act
memory_bytes = Compiler(bytecore_assembly).get_memory_bytes()
byte_core = ByteCore(memory_bytes)
byte_core.cycle_until_halt()
dump = byte_core.dump()

# Assert
pc_msb = dump.pc_msb_register.value
pc_lsb = dump.pc_lsb_register.value
pc = pc_msb * 256 + pc_lsb
assert pc == int('FF_06', 16)

accumulator = dump.accumulator.value
assert accumulator == expected

actual = dump.memory[int('FF_FF', 16)].value
assert actual == expected

## Advanced Example Program

In [3]:
advanced_program_bytecore_assembly = """
FF 00 10 ; JMP
FF 01 20

FF 10 37 ; 55
FF 11 14 ; 20
FF 12 02 ;  2
FF 13 01 ;  1

FF 20 01 ; LOAD
FF 21 10
FF 22 04 ; ADD
FF 23 12
FF 24 02 ; STORE
FF 25 10
FF 26 01 ; LOAD
FF 27 11
FF 28 08 ; SUB
FF 29 13
FF 2A 02 ; STORE
FF 2B 11
FF 2C 01 ; LOAD
FF 2D 11
FF 2E 20 ; JZ
FF 2F 80
FF 30 10 ; JMP
FF 31 20

FF 80 01 ; LOAD
FF 81 10
FF 82 02 ; STORE
FF 83 FF
FF 84 00 ; HALT
"""

# Arrange
expected = 95
bytecore_assembly = '\n'.join(
    [bytecorebyte_bytecore_assembly, advanced_program_bytecore_assembly])

# Act
memory_bytes = Compiler(bytecore_assembly).get_memory_bytes()
byte_core = ByteCore(memory_bytes)
byte_core.cycle_until_halt()
dump = byte_core.dump()

# Assert
pc_msb = dump.pc_msb_register.value
pc_lsb = dump.pc_lsb_register.value
pc = pc_msb * 256 + pc_lsb
assert pc == int('FF_84', 16)

accumulator = dump.accumulator.value
assert accumulator == expected

actual = dump.memory[int('FF_FF', 16)].value
assert actual == expected

## License

This project is licensed under the terms of the MIT License. See the [LICENSE](https://github.com/joakimwinum/bytecore-byte/blob/main/LICENSE) file for the full text.